# 3.9 - Traduzindo a análise para um raspador

Nesta aula são abordados os seguintes temas:
1. Seletores com Scrapy;
2. Iteração com os elementos com *for*;
3. Tradução da análise de página web para um raspador com Scrapy.

## Configurando o ambiente

Antes de começar, é preciso fazer um *fork* e *clone* do [repositório do Querido Diário](https://github.com/okfn-brasil/querido-diario). Depois, é preciso configurar o ambiente executando os códigos abaixo.

In [ ]:
# criar um ambiente virtual
python -m venv .venv

In [ ]:
# ativar o ambiente virtual
.venv/Scripts/activate

In [ ]:
# instalar as bibliotecas requeridas
pip install -r data_collection/requirements-dev.txt 

In [ ]:
# instalar o pré-commit, uma ferramenta que, ao fazer o commit do código, verifica se ele se adequa aos padrões do projeto
pip install pre-commit

Para Depois de configurado o ambiente, criaremos um arquivo chamado *to_porto_nacional.py* dentro da pasta *data_collection/spiders*. Neste arquivo criaremos o raspador para a página do [**Diário Oficial de Porto Nacional (TO)**](https://diariooficial.portonacional.to.gov.br/edicoes).

## 1. Seletores com Srapy

Para ajustar a configuração inicial do rapspador, serão importadas as biblitecas usadas, bem como definindo a classe do mesmo utilizando os comandos abaixo:

In [ ]:
import datetime # biblioteca para manipular datas
import re # biblioteca para manipular regex
import scrapy
from gazette.items import Gazette # 
from gazette.spiders.base import BaseGazetteSpider # importando classe básica para trabalhar o raspador

class ToPortoNacionalSpider(BaseGazetteSpider): # definindo a classe para o raspador do diário oficial de Porto Nacional
    name = "to_porto_nacional" # nome do raspador
    TERRITORY_ID = "1718204" # id da cidade segundo o IBGE
    allowed_domains = ["diariooficial.portonacional.to.gov.br"]
    start_date = datetime.date(2017, 4, 3) # data inicial da raspagem
    start_url = ["https://www.diariooficial.portonacional.to.gov.br/edicoes"] # url de partida do raspador

## 2. Iteração com os elementos com o for

A partir dessa configuração inicial, iremos construir uma função que leia linha por linha da tabela de edições do Diário Oficial.

In [ ]:
def parse(self, response):  # definição da função parse que vai receber o url
            
    editions = response.xpath('//table[@class="table"]/tbody/tr') # definição da variável que vai encontrar a tabela onde estão as edições do diário oficial

    for edition in editions: # for para cada linha da tabela definida em editions
        link = edition.xpath('./td[3]/div/a/@href').get() # recebe o link edição em pdf vinda do href
        title = edition.xpath('./td[1]/strong/text()').get() # recebe o título da edição vinda do texto em strong
        edition_number = re.search(r'EDIÇÃO Nº (\d+)', title).group(1) # recebe o número da edição vindo da busca no título da edição
        date_text = edition.xpath('./td[1]/br/following-sibling::text()').get()  # recebe a data vinda do texto após o br
        day, month, year = re.match(r"\s+(\d+) de (\w+) de (\d+)\s+", date_text).group(1, 2, 3) # recebe dia, mês e ano da busca feita na data
        date = year+month+day # recebe a data vinda da junção de ano, mês e dia
        date = datetime.datetime.strptime(date, "%Y%B%d").date() # transforma a data em datetime

## 3. Tradução para o Scrapy

Depois, vamos guardar o que encontramos em cada linha, fazemos a comparação entre a data da edição raspada e a data de partida do raspador e utilizamos o tipo de retorno *yield* para persistir os dados utilizando a classe *Gazette*.

In [ ]:
if date < self.start_date: # compara a última data com a data inicial defininida como argumento da classe
    return
            
yield Gazette( # retorna as informações abaixo em yield para cada linha da tabela
    date=date,
    edition_number=edition_number, 
    file_urls=[link], 
    power="executive",
    )

Por fim, fazemos a paginação utilizando o XPath do link para próxima página.

In [83]:
next_page_link = response.xpath("//a[@rel="next"]/@href").get() # recebe a url da próxima página
if next_page_link: # caso realmente exista o link
    yield scrapy.Request( # faz nova requisição
        url=response.urljoin(next_page_link), # coloca a próxima página na resposta
        callback=self.parse, # inicia novamente o método parse
    )

2023-07-12 00:00:00
